In [1]:
def clipAlpha(aj,H,L):
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

def selectJrand(i,m):
    j=i #we want to select any J not equal to i
    while (j==i):
        j = int(np.random.uniform(0,m))
    return j
def kernelTrans(X, A, kTup): #calc the kernel or transform data to a higher dimensional space
    m,n = np.shape(X)
    K = np.mat(np.zeros((m,1)))
    if kTup[0]=='linear': K = X * A.T   #linear kernel
    elif kTup[0]=='rbf':
        for j in range(m):
            deltaRow = np.mat(X[j,:] - A)
            K[j] = deltaRow*deltaRow.T
        K = np.exp(K/(-1*kTup[1]**2)) #divide in NumPy is element-wise not matrix like Matlab
    else: 
        raise NameError('Houston We Have a Problem That Kernel is not recognized')
    return np.mat(K)

class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):  # Initialize the structure with the parameters 
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m,1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m,2))) #first column is valid flag
        self.K = np.mat(np.zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)
        
def calcEk(oS, k):
    fXk = float(np.multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek
        
def selectJ(i, oS, Ei):         #this is the second choice -heurstic, and calcs Ej
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]  #set valid #choose the alpha that gives the maximum delta E
    validEcacheList = np.nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:   #loop through valid Ecache values and find the one that maximizes delta E
            if k == i: continue #don't calc for i, waste of time
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:   #in this case (first time around) we don't have any valid eCache values
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):#after any alpha has changed update the new value in the cache
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1,Ek]
        
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei) #this has been changed from selectJrand
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: 
            #print "L==H"; 
            return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j] #changed for kernel
        if eta >= 0: 
            #print "eta>=0"; 
            return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j) #added this for the Ecache
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): 
            #print "j not moving enough"; 
            return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
        updateEk(oS, i) #added this for the Ecache                    #the update is in the oppostie direction
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: 
        return 0

def smoP(dataMatIn, classLabels, C, toler, maxIter,kTup=('linear', 0)):    #full Platt SMO
    oS = optStruct(np.mat(dataMatIn),np.mat(classLabels).transpose(),C,toler, kTup)
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:   #go over all
            for i in range(oS.m):        
                alphaPairsChanged += innerL(i,oS)
                #print "fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
            iter += 1
        else:#go over non-bound (railed) alphas
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                #print "non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
            iter += 1
        if entireSet: entireSet = False #toggle entire set loop
        elif (alphaPairsChanged == 0): entireSet = True  
        #print "iteration number: %d" % iter
    return oS.b,oS.alphas 
def calcWs(alphas,dataArr,classLabels):
    X = np.mat(dataArr); 
    labelMat = np.mat(classLabels).transpose()
    m,n = np.shape(X)
    w = np.zeros((n,1))
    for i in range(m):
        w += np.multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

In [2]:
class SVM:
    def __init__(self,C=1,gamma=1,kernel='linear',max_iter=2000,tol=1e-3):
        self.c=C
        self.kernel=kernel
        self.max_iter=max_iter
        self.tol=tol
        self.gamma=1
        self.kTup=(self.kernel,self.gamma)
        
    def fit(self,X,y):
        y=2*y-1
        self.y=y
        self.b,self.alphas=smoP(X,y,C=self.c,toler=self.tol,maxIter=self.max_iter,kTup=self.kTup)
        self.w=calcWs(self.alphas,X,y).flatten()
        self.support_index_=np.where(self.alphas>0)[0]
        self.n_support_=len(self.support_index_)
        self.support_vectors_=X[self.support_index_]
        self.support_labels_=self.y[self.support_index_]
        self.b=float((1/y[self.support_index_]-X[self.support_index_]@self.w)[0])
        return self
        
        
    def predict(self,X):
        m,n = X.shape
        r=[]
        for i in range(m):
            K = kernelTrans(self.support_vectors_,X[i],self.kTup)
            predict=(np.sign(np.sum(K.T.A*(np.multiply(self.support_labels_,self.alphas[self.support_index_].T.A))) + self.b)+1)/2
            r.append(predict)
        return np.array(r)      

In [3]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_gaussian_quantiles
X,y=load_breast_cancer().data,load_breast_cancer().target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=7)
np.random.seed(88)

my_lrsvm=SVM(C=1,kernel='linear').fit(X_train,y_train)
accuracy_score(y_test,my_lrsvm.predict(X_test))

0.8947368421052632

In [5]:
my_lrsvm.n_support_

6

In [6]:
my_lrsvm.b

9.055606193278665

In [7]:
my_lrsvm.support_index_

array([  0,   1,  78, 131, 217, 294], dtype=int64)

In [8]:
my_lrsvm.support_vectors_.shape

(6, 30)

In [12]:
my_lrsvm.alphas[my_lrsvm.support_index_].T.A

array([[9.23519234e-05, 4.52117925e-05, 6.33717438e-06, 7.62368789e-06,
        1.31384851e-05, 2.09566551e-05]])

In [17]:
X,y=make_gaussian_quantiles(n_classes=2,n_samples=500,random_state=7)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=7)
for i in np.linspace(0.1,1,10):
    my_svm=SVM(C=1,kernel='rbf',gamma=i).fit(X_train,y_train)
    print(accuracy_score(y_test,my_svm.predict(X_test)))

0.48
0.74
0.74
0.92
0.74
0.48
0.92
0.89
0.74
0.48


In [14]:
my_svm=SVM(C=1,kernel='linear',gamma=1).fit(X_train,y_train)
accuracy_score(y_test,my_svm.predict(X_test))

0.46